In [46]:
import torch
import torch.nn.functional as F
import optuna
import matplotlib.pyplot as plt

In [47]:
# read in all the words
words = open('makemore/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [48]:
# build the dataset

def build_dataset(words, block_size):
    X, Y = [], []
    for w in words:
    
        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [49]:
def train_model(steps, batch_size, learning_rate, l1_num_inputs, x, y, stepi, lossi):
    for p in parameters:
        p.requires_grad = True

    for i in range(steps):
        ix = torch.randint(0, x.shape[0], (batch_size,))
        
        # Forward pass
        emb = C[x[ix]]  # Shape: (batch_size, block_size, embedding_dims)
        emb = emb.view(batch_size, -1)  # Flatten properly
        
        h = torch.tanh(emb @ W1 + b1)  # Now matches shape (batch_size, l2_num_neurons)
        logits = h @ W2 + b2  # Shape: (batch_size, 27)
        loss = F.cross_entropy(logits, y[ix])
        
        # Backward pass
        for p in parameters:
            p.grad = None
        loss.backward()

        for p in parameters:
            p.data += -learning_rate * p.grad

        stepi.append(i)
        lossi.append(loss.log10().item())


def total_loss(x, y, l1_num_inputs):
    emb = C[x] # (batch_size, block_size, l1_num_inputs)
    h = torch.tanh(emb.view(-1, l1_num_inputs) @ W1 + b1) # (l1_num_inputs, l2_num_neurons)
    logits = h @ W2 + b2 # (batch_size, 27)
    loss = F.cross_entropy(logits, y)
    return loss.item()

In [50]:
def get_samples(num_samples):
    # sample from the model
    g = torch.Generator().manual_seed(2147483647 + 10)
    
    for _ in range(20):
    
        out = []
        context = [0] * block_size # initialize with all ...
        while True:
            emb = C[torch.tensor([context])] # (1, block_size, d)
            h = torch.tanh(emb.view(1, -1) @ W1 + b1)
            logits = h @ W2 + b2
            probs = F.softmax(logits, dim=1)
            ix = torch.multinomial(probs, num_samples=1, generator=g).item()
            context = context[1:] + [ix]
            out.append(ix)
            if ix == 0:
                break
    
        print(''.join(itos[i] for i in out))

In [55]:
def objective(trial):
    # Suggest hyperparameters
    block_size = trial.suggest_int("block_size", 8, 12)  # Context length
    embedding_dims = trial.suggest_int("embedding_dims", 8, 12)  # Embedding size
    l2_num_neurons = trial.suggest_int("l2_num_neurons", 50, 150)  # Hidden layer size
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])  # Batch size
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 1e-1, log=True)  # Learning rate

    # Build datasets with new block_size
    Xtr, Ytr = build_dataset(words[:n1], block_size)
    Xdev, Ydev = build_dataset(words[n1:n2], block_size)

    # Define model parameters
    l1_num_inputs = embedding_dims * block_size
    g = torch.Generator().manual_seed(2147483647)
    
    global C, W1, b1, W2, b2  # Ensure variables are accessible
    C = torch.randn((27, embedding_dims), generator=g)
    W1 = torch.randn((l1_num_inputs, l2_num_neurons), generator=g)
    b1 = torch.randn(l2_num_neurons, generator=g)
    W2 = torch.randn((l2_num_neurons, 27), generator=g)
    b2 = torch.randn(27, generator=g)
    
    parameters = [C, W1, b1, W2, b2]
    for p in parameters:
        p.requires_grad = True

    # Training loop
    for i in range(50000):  # Reduce iterations for speed
        ix = torch.randint(0, Xtr.shape[0], (batch_size,))
        emb = C[Xtr[ix]]
        h = torch.tanh(emb.view(-1, l1_num_inputs) @ W1 + b1)
        logits = h @ W2 + b2
        loss = F.cross_entropy(logits, Ytr[ix])

        # Backprop
        for p in parameters:
            p.grad = None
        loss.backward()

        for p in parameters:
            p.data += -learning_rate * p.grad

    # Compute validation loss
    val_loss = total_loss(Xdev, Ydev, l1_num_inputs)
    
    return val_loss  # Optuna minimizes by default

In [56]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)  # Try 30 different sets of hyperparameters

print("Best hyperparameters:", study.best_params)

[I 2025-03-01 15:05:28,172] A new study created in memory with name: no-name-55c65799-95c8-4a8b-8db3-ac793ef8ab40


torch.Size([182778, 10]) torch.Size([182778])
torch.Size([22633, 10]) torch.Size([22633])


[I 2025-03-01 15:05:35,737] Trial 0 finished with value: 2.6170952320098877 and parameters: {'block_size': 10, 'embedding_dims': 10, 'l2_num_neurons': 60, 'batch_size': 32, 'learning_rate': 0.050675099941790085}. Best is trial 0 with value: 2.6170952320098877.


torch.Size([182778, 10]) torch.Size([182778])
torch.Size([22633, 10]) torch.Size([22633])


[I 2025-03-01 15:05:44,663] Trial 1 finished with value: 2.6354153156280518 and parameters: {'block_size': 10, 'embedding_dims': 9, 'l2_num_neurons': 148, 'batch_size': 64, 'learning_rate': 0.0668998711142591}. Best is trial 0 with value: 2.6170952320098877.


...

torch.Size([182778, 8]) torch.Size([182778])
torch.Size([22633, 8]) torch.Size([22633])


[I 2025-03-01 15:09:57,788] Trial 31 finished with value: 2.514582872390747 and parameters: {'block_size': 8, 'embedding_dims': 11, 'l2_num_neurons': 100, 'batch_size': 64, 'learning_rate': 0.06515495056860778}. Best is trial 31 with value: 2.514582872390747.


torch.Size([182778, 8]) torch.Size([182778])
torch.Size([22633, 8]) torch.Size([22633])


[W 2025-03-01 15:10:05,404] Trial 32 failed with parameters: {'block_size': 8, 'embedding_dims': 11, 'l2_num_neurons': 100, 'batch_size': 64, 'learning_rate': 0.0639699936805835} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/8_/kl72rt_d67z2yxqnyg9_n3mw0000gn/T/ipykernel_25469/175724720.py", line 39, in objective
    loss.backward()
    ~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/torch/_tensor.py", line 626, in backward
    torch.autograd.backward(
    ~~~~~~~~~~~~~~~~~~~~~~~^
        self, gradient, retain_graph, create_graph, inputs=inputs
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages/torch

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params  # Get the best hyperparams from Optuna
print("Best Hyperparameters:", best_params)

block_size = best_params["block_size"]
embedding_dims = best_params["embedding_dims"]
l2_num_neurons = best_params["l2_num_neurons"]

l1_num_inputs = embedding_dims * block_size

# Reinitialize parameters using the best hyperparameters
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, embedding_dims), generator=g)
W1 = torch.randn((l1_num_inputs, l2_num_neurons), generator=g)
b1 = torch.randn(l2_num_neurons, generator=g)
W2 = torch.randn((l2_num_neurons, 27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [ ]:
Xtr, Ytr = build_dataset(words[:n1], block_size)
Xdev, Ydev = build_dataset(words[n1:n2], block_size)
Xte, Yte = build_dataset(words[n2:], block_size)

In [ ]:
stepi = []
lossi = []
train_model(
    steps=100000,
    batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    l1_num_inputs=l1_num_inputs,
    x=Xtr,
    y=Ytr,
    stepi=stepi,
    lossi=lossi
)
plt.plot(stepi, lossi)

In [ ]:
print("Training loss:", total_loss(Xtr, Ytr, l1_num_inputs))
print("Validation loss:", total_loss(Xdev, Ydev, l1_num_inputs))
print("Test loss:", total_loss(Xte, Yte, l1_num_inputs))

In [ ]:
get_samples(20)